In [2]:
using DataFrames, TableOperations, Tables
using Recommenders: Movielens100k, load_dataset, leave_one_out_split, ItemkNN, evaluate_u2i, PrecisionAtK

In [3]:
ml100k = Movielens100k()
download(ml100k)
rating, user, movie = load_dataset(ml100k);

In [4]:
rating = rating |> TableOperations.filter(x->Tables.getcolumn(x, :rating) >= 4)

TableOperations.Filter{var"#3#4", CSV.File{false}}(var"#3#4"(), CSV.File("/Users/keisuke.yanagi/workspace/Recommenders.jl/src/dataset/../../dataset/movielens100k/u.data"):
Size: 100000 x 4
Tables.Schema:
 :userid     Int64
 :movieid    Int64
 :rating     Int64
 :timestamp  Int64)

In [5]:
train_valid_table, test_table = leave_one_out_split(rating)
train_table, valid_table = leave_one_out_split(train_valid_table)
length(Tables.rows(train_table)), length(Tables.rows(valid_table)), length(Tables.rows(test_table))


(51960, 1643, 1772)

In [6]:
metric = PrecisionAtK(10)

PrecisionAtK(10)

In [7]:
using TreeParzen

In [8]:
space = Dict(
    :topk=>HP.QuantUniform(:topk, 10., 500., 1.),
    :shrink=>HP.LogUniform(:shrink, log(1e-3), log(1e3)),
    :weighting=>HP.Choice(:weighting, 
        [
            Dict(:weighting=>:dummy, :weighting_at_inference=>false),
            Dict(:weighting=>:tfidf, :weighting_at_inference=>false),
            Dict(:weighting=>:bm25, :weighting_at_inference=>HP.Choice(:weighting_at_inference, [true, false]))
        ]
    ),
    :normalize=>HP.Choice(:normalize, [true, false])
)

Dict{Symbol, TreeParzen.Types.AbstractDelayed} with 4 entries:
  :weighting => Choice(Param(:weighting, RandIndex(3)), Dict{Symbol, Any}[Dict(…
  :topk      => QuantUniform(:topk, QuantUniform(10.0, 500.0, 1.0))
  :normalize => Choice(Param(:normalize, RandIndex(2)), Bool[1, 0])
  :shrink    => LogUniform(:shrink, LogUniform(-6.90776, 6.90776))

In [9]:
function invert_output(params)
    k = convert(Int, params[:topk])
    model = ItemkNN(k, params[:shrink],params[:weighting][:weighting],params[:weighting][:weighting_at_inference],params[:normalize])
    result = evaluate_u2i(model, train_table, valid_table, metric, 10, col_user=:userid, col_item=:movieid, col_rating=:rating, drop_history=false)
    @show params, result
    return -result
end

invert_output (generic function with 1 method)

In [10]:
best = fmin(invert_output, space, 20, logging_interval=-1)

(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :bm25, :weighting_at_inference => true), :topk => 402.0, :normalize => false, :shrink => 8.474651140280114), 0.01115834218916045)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :tfidf, :weighting_at_inference => false), :topk => 444.0, :normalize => false, :shrink => 41.30711249679659), 0.013815090329436747)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :tfidf, :weighting_at_inference => false), :topk => 236.0, :normalize => false, :shrink => 7.554383231211263), 0.013815090329436747)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :bm25, :weighting_at_inference => false), :topk => 483.0, :normalize => false, :shrink => 122.73225840814129), 0.014558979808714108)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk =>

┌ Info: fmin: 20 / 20 trials carried out
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:176
┌ Info: Successfully completed fmin 
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:231


Dict{Symbol, Any} with 4 entries:
  :weighting => Dict{Symbol, Any}(:weighting=>:dummy, :weighting_at_inference=>…
  :topk      => 256.0
  :normalize => true
  :shrink    => 3.64117

In [11]:
best_model = ItemkNN(convert(Int, best[:topk]), best[:shrink],best[:weighting][:weighting],best[:weighting][:weighting_at_inference],best[:normalize])
evaluate_u2i(best_model, train_valid_table, test_table, metric, 10, col_user=:userid, col_item=:movieid, col_rating=:rating)

0.02006369426751593